In [1]:
import numpy as np
import matplotlib.pyplot as plt

from qdlutils.hardware.nidaq.synchronous.nidaqsequencer import NidaqSequencer
from qdlutils.hardware.nidaq.synchronous.nidaqsequencerinput import *
from qdlutils.hardware.nidaq.synchronous.nidaqsequenceroutput import *

from qdlutils.applications.qdlple2.application_controller import PLEControllerPulsedRepumpSegmented

Define the inputs and outputs.

In [2]:
inputs = {
    'photodiode' : NidaqSequencerAIVoltage(
            name = 'photodiode',
            device = 'Dev1',
            channel = 'ai1',
    ),
    'counter' : NidaqSequencerCIEdge(
            name = 'counter',
            device = 'Dev1',
            channel = 'ctr2',
            terminal = 'PFI0',
    )
}

outputs = {
    'scan_laser' : NidaqSequencerAOVoltage(
            name = 'scan_laser',
            device = 'Dev1',
            channel = 'ao3',
            min_voltage = -3,
            max_voltage = 5
    ),
    'repump_laser' : NidaqSequencerAOVoltage(
            name = 'repump_laser',
            device = 'Dev2',
            channel = 'ao0',
            min_voltage = -5,
            max_voltage = 5
    )
}

process_instructions = {} # Just take the first sample point for all

Create the controller

In [3]:
controller = PLEControllerPulsedRepumpSegmented(
        inputs = inputs,
        outputs = outputs,
        scan_laser_id = 'scan_laser',
        repump_laser_id = 'repump_laser',
        counter_id = 'counter',
        clock_device = 'Dev1',
        clock_channel = 'port0',
        process_instructions = process_instructions
)

In [4]:
controller.configure_sequence(
        min = -3,
        max = 5,
        n_pixels_up = 200,
        n_pixels_down = 10,
        n_subpixels = 1,
        time_up = 10,
        time_down = 2,
        time_repump = 0,
)

In [5]:
data = controller._run_sequence(
    process_method=controller.process_data, 
    process_kwargs={'instructions': process_instructions})

starting upscan
waiting for  photodiode
waiting for  counter
waiting for  scan_laser
waiting for  repump_laser


DaqError: Wait Until Done did not indicate that the task was done within the specified timeout.
Increase the timeout, check the program, and make sure connections for external timing and triggering are in place.
Task Name: _unnamedTask<6>

Status Code: -200560

In [7]:
controller.__dict__

{'inputs': {'photodiode': <qdlutils.hardware.nidaq.synchronous.nidaqsequencerinput.NidaqSequencerAIVoltage at 0x1c9690af220>,
  'counter': <qdlutils.hardware.nidaq.synchronous.nidaqsequencerinput.NidaqSequencerCIEdge at 0x1c9690af640>},
 'outputs': {'scan_laser': <qdlutils.hardware.nidaq.synchronous.nidaqsequenceroutput.NidaqSequencerAOVoltage at 0x1c9690af4c0>,
  'repump_laser': <qdlutils.hardware.nidaq.synchronous.nidaqsequenceroutput.NidaqSequencerAOVoltage at 0x1c9690af040>},
 'clock_device': 'Dev1',
 'clock_channel': 'port0',
 'repump_sequencer': <qdlutils.hardware.nidaq.synchronous.nidaqsequencer.NidaqSequencer at 0x1c9690af9d0>,
 'upscan_sequencer': <qdlutils.hardware.nidaq.synchronous.nidaqsequencer.NidaqSequencer at 0x1c9755aae20>,
 'downscan_sequencer': <qdlutils.hardware.nidaq.synchronous.nidaqsequencer.NidaqSequencer at 0x1c9690afd60>,
 'sequence_settings': {'min': -3,
  'max': 5,
  'n_pixels_up': 200,
  'n_pixels_down': 10,
  'n_subpixels': 1,
  'n_subpixels_up': 200,
  'n